# Unit 5 Lecture 2 - Wrangling Data

ESI4628: Decision Support Systems for Industrial Engineers<br>
University of Central Florida
Dr. Ivan Garibay, Ramya Akula, Mostafa Saeidi, Madeline Schiappa, and Brett Belcher. 
https://github.com/igaribay/DSSwithPython/blob/master/DSS-Week05/Notebook/DSS-Unit05-Lecture02.2018.ipynb

## Notebook Learning Objectives
After studying this notebook students should be able to:
- Reshaping data using stack and unstack index data hierarchically
- Group, aggregate, and transform data
- merge datasets using inner, outer, left and right join operations
- create Pivot tables
- remove NaN from data and remove duplicates

# Overview

When the large amounts of data are spread across various files, accessing that data is lot easier when organized properly. In this chapter we will learn how to aggregate the data by wrangling as needed. 

In [10]:
import pandas as pd
import numpy as np

# Hierarchical Indexing

Dealing with higher dimensional data is always a challenge, so we need to index them hierarchically, this helps in working with higher dimensional data in lower dimensional form, also known as hierarchical indexing. Lets begin with an simple example. 

In [12]:
pd.read_excel('../Data/DSS-Unit05-File01.xlsx', sheet_name = 'Sales') #importing data from an Excel file

Apple  Amazon  Alphabet
2018-06-30  53427   52886     32758
2018-03-31  61224   51042     31393
2017-12-31  88477   60453     32521
2017-09-30  52574   43744     27963
2017-06-30  45260   37955     26007

To create a hierarchical index, we simply define the index of the dataframe as two or more lists as follows:

In [30]:
data = pd.DataFrame(np.random.randn(27).reshape(9,3), index = [['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd' ], [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

0         1         2
a 1 -0.561031  0.182514  0.934103
  2  1.174967  2.440814  0.094836
  3  0.265001  0.347549 -0.750255
b 1 -0.555708  2.614888 -0.636853
  3  1.272148  0.543570  1.761683
c 1  1.355795  1.143298  1.112891
  2  0.659473 -2.144723  1.581474
d 2  0.119811 -0.632336 -0.261156
  3  0.938050 -0.490501 -0.376920

A hierarchical index can be also at the column indexes:

In [75]:
data = pd.DataFrame(np.random.randn(27).reshape(9,3), index = [['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd' ], [1, 2, 3, 1, 3, 1, 2, 2, 3]], columns = [['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
data

Ohio            Colorado
        Green       Red     Green
a 1 -1.351198 -0.142386  0.807120
  2 -1.392440 -0.514279 -0.129106
  3 -0.658959 -0.226176 -0.796778
b 1  0.887053  1.467681  0.941326
  3  0.416897 -1.116810 -1.073804
c 1  1.014618 -1.736553 -0.968988
  2  1.004606  0.323400 -0.162310
d 2  0.273868  1.415743 -3.284162
  3 -0.580739  0.943904 -0.489641

# Reshaping Data Frames

We use <code>.stack()</code> and <code>.unstack()</code> methods to reshape the Data Frame by exchanging row for column indexes. Lets load the following Excel file containing financial data per quarter of three big tech firms to illustrate.
** Note: ** I am using <code>.read_excel()</code> to read the Excel file. I am providing _Excel file location_, _sheet name inside the file_, and that _first and second rows are headers_ not data.

## <code>.stack()</code> and <code>.unstack()</code>

In [36]:
findata = pd.read_excel('../Data/DSS-Unit05-File01.xlsx', sheet_name = 'Financials_v1', header = [0,1]) #importing data from an Excel file
findata

Sales/Revenue                 Gross Income/Profit                  \
                   Apple Amazon Alphabet               Apple Amazon Alphabet   
2018-06-30         53427  52886    32758               20789  22254    18875   
2018-03-31         61224  51042    31393               23530  20307    17926   
2017-12-31         88477  60453    32521               34069  21959    18254   
2017-09-30         52574  43744    27963               19902  16195    16815   
2017-06-30         45260  37955    26007               17267  14504    15634   

           Total Assets                  Total Liabilities                   
                  Apple  Amazon Alphabet             Apple  Amazon Alphabet  
2018-06-30       375319  134100   197295            241272   99105    44793  
2018-03-31       321686  126362   167497            193437   94899    28461  
2017-12-31       290479  131310   147461            171124  103601    27130  
2017-09-30       231839  115267   129187            120292   90609    25327  
2017-06-30       207000   87781   110920             83451   64567    23611

Now lets use <code>.stack()</code> and see what happens:

In [47]:
stacked_findata = findata.stack()
stacked_findata

Gross Income/Profit  Sales/Revenue  Total Assets  \
2018-06-30 Alphabet                18875          32758        197295   
           Amazon                  22254          52886        134100   
           Apple                   20789          53427        375319   
2018-03-31 Alphabet                17926          31393        167497   
           Amazon                  20307          51042        126362   
           Apple                   23530          61224        321686   
2017-12-31 Alphabet                18254          32521        147461   
           Amazon                  21959          60453        131310   
           Apple                   34069          88477        290479   
2017-09-30 Alphabet                16815          27963        129187   
           Amazon                  16195          43744        115267   
           Apple                   19902          52574        231839   
2017-06-30 Alphabet                15634          26007        110920   
           Amazon                  14504          37955         87781   
           Apple                   17267          45260        207000   

                     Total Liabilities  
2018-06-30 Alphabet              44793  
           Amazon                99105  
           Apple                241272  
2018-03-31 Alphabet              28461  
           Amazon                94899  
           Apple                193437  
2017-12-31 Alphabet              27130  
           Amazon               103601  
           Apple                171124  
2017-09-30 Alphabet              25327  
           Amazon                90609  
           Apple                120292  
2017-06-30 Alphabet              23611  
           Amazon                64567  
           Apple                 83451

The inner most column index (Apple, Amazon, Alphabet) was transposed from columns into rows. Now lets try to reverse this operation with <code>.unstack()</code>

In [49]:
stacked_findata.unstack() # operation is reversible

Gross Income/Profit               Sales/Revenue                \
                      Alphabet Amazon  Apple      Alphabet Amazon  Apple   
2018-06-30               18875  22254  20789         32758  52886  53427   
2018-03-31               17926  20307  23530         31393  51042  61224   
2017-12-31               18254  21959  34069         32521  60453  88477   
2017-09-30               16815  16195  19902         27963  43744  52574   
2017-06-30               15634  14504  17267         26007  37955  45260   

           Total Assets                 Total Liabilities                  
               Alphabet  Amazon   Apple          Alphabet  Amazon   Apple  
2018-06-30       197295  134100  375319             44793   99105  241272  
2018-03-31       167497  126362  321686             28461   94899  193437  
2017-12-31       147461  131310  290479             27130  103601  171124  
2017-09-30       129187  115267  231839             25327   90609  120292  
2017-06-30       110920   87781  207000             23611   64567   83451

Lets load another example from the same Excel file. This example adds a third index for columns:

In [70]:
findata2 = pd.read_excel('../Data/DSS-Unit05-File01.xlsx', sheet_name = 'Financials_v3', header = [0,1,2]) #importing data from an Excel file
findata2

Income Statement                                             \
              Sales/Revenue                 Gross Income/Profit          
                      Apple Amazon Alphabet               Apple Amazon   
2018-06-30            53427  52886    32758               20789  22254   
2018-03-31            61224  51042    31393               23530  20307   
2017-12-31            88477  60453    32521               34069  21959   
2017-09-30            52574  43744    27963               19902  16195   
2017-06-30            45260  37955    26007               17267  14504   

                    Balance Sheet                                             \
                     Total Assets                  Total Liabilities           
           Alphabet         Apple  Amazon Alphabet             Apple  Amazon   
2018-06-30    18875        375319  134100   197295            241272   99105   
2018-03-31    17926        321686  126362   167497            193437   94899   
2017-12-31    18254        290479  131310   147461            171124  103601   
2017-09-30    16815        231839  115267   129187            120292   90609   
2017-06-30    15634        207000   87781   110920             83451   64567   

                     
                     
           Alphabet  
2018-06-30    44793  
2018-03-31    28461  
2017-12-31    27130  
2017-09-30    25327  
2017-06-30    23611

In [55]:
findata2.stack()

Balance Sheet                      Income Statement  \
                     Total Assets Total Liabilities Gross Income/Profit   
2018-06-30 Alphabet        197295             44793               18875   
           Amazon          134100             99105               22254   
           Apple           375319            241272               20789   
2018-03-31 Alphabet        167497             28461               17926   
           Amazon          126362             94899               20307   
           Apple           321686            193437               23530   
2017-12-31 Alphabet        147461             27130               18254   
           Amazon          131310            103601               21959   
           Apple           290479            171124               34069   
2017-09-30 Alphabet        129187             25327               16815   
           Amazon          115267             90609               16195   
           Apple           231839            120292               19902   
2017-06-30 Alphabet        110920             23611               15634   
           Amazon           87781             64567               14504   
           Apple           207000             83451               17267   

                                   
                    Sales/Revenue  
2018-06-30 Alphabet         32758  
           Amazon           52886  
           Apple            53427  
2018-03-31 Alphabet         31393  
           Amazon           51042  
           Apple            61224  
2017-12-31 Alphabet         32521  
           Amazon           60453  
           Apple            88477  
2017-09-30 Alphabet         27963  
           Amazon           43744  
           Apple            52574  
2017-06-30 Alphabet         26007  
           Amazon           37955  
           Apple            45260

In [58]:
findata2.stack(level=2) #level 2 is the "inner-most" default, so we obtain the same result than above

Balance Sheet                      Income Statement  \
                     Total Assets Total Liabilities Gross Income/Profit   
2018-06-30 Alphabet        197295             44793               18875   
           Amazon          134100             99105               22254   
           Apple           375319            241272               20789   
2018-03-31 Alphabet        167497             28461               17926   
           Amazon          126362             94899               20307   
           Apple           321686            193437               23530   
2017-12-31 Alphabet        147461             27130               18254   
           Amazon          131310            103601               21959   
           Apple           290479            171124               34069   
2017-09-30 Alphabet        129187             25327               16815   
           Amazon          115267             90609               16195   
           Apple           231839            120292               19902   
2017-06-30 Alphabet        110920             23611               15634   
           Amazon           87781             64567               14504   
           Apple           207000             83451               17267   

                                   
                    Sales/Revenue  
2018-06-30 Alphabet         32758  
           Amazon           52886  
           Apple            53427  
2018-03-31 Alphabet         31393  
           Amazon           51042  
           Apple            61224  
2017-12-31 Alphabet         32521  
           Amazon           60453  
           Apple            88477  
2017-09-30 Alphabet         27963  
           Amazon           43744  
           Apple            52574  
2017-06-30 Alphabet         26007  
           Amazon           37955  
           Apple            45260

In [69]:
findata2.stack(level=0) #level 0 is the "financial report" level, this time that level gets "stacked"

Alphabet    Amazon     Apple
2018-06-30 Gross Income/Profit Income Statement   18875.0   22254.0   20789.0
           Sales/Revenue       Income Statement   32758.0   52886.0   53427.0
           Total Assets        Balance Sheet     197295.0  134100.0  375319.0
           Total Liabilities   Balance Sheet      44793.0   99105.0  241272.0
2018-03-31 Gross Income/Profit Income Statement   17926.0   20307.0   23530.0
           Sales/Revenue       Income Statement   31393.0   51042.0   61224.0
           Total Assets        Balance Sheet     167497.0  126362.0  321686.0
           Total Liabilities   Balance Sheet      28461.0   94899.0  193437.0
2017-12-31 Gross Income/Profit Income Statement   18254.0   21959.0   34069.0
           Sales/Revenue       Income Statement   32521.0   60453.0   88477.0
           Total Assets        Balance Sheet     147461.0  131310.0  290479.0
           Total Liabilities   Balance Sheet      27130.0  103601.0  171124.0
2017-09-30 Gross Income/Profit Income Statement   16815.0   16195.0   19902.0
           Sales/Revenue       Income Statement   27963.0   43744.0   52574.0
           Total Assets        Balance Sheet     129187.0  115267.0  231839.0
           Total Liabilities   Balance Sheet      25327.0   90609.0  120292.0
2017-06-30 Gross Income/Profit Income Statement   15634.0   14504.0   17267.0
           Sales/Revenue       Income Statement   26007.0   37955.0   45260.0
           Total Assets        Balance Sheet     110920.0   87781.0  207000.0
           Total Liabilities   Balance Sheet      23611.0   64567.0   83451.0

In [60]:
findata2.stack().stack() # lets stack twice, two inner-most levels get stacked.

Balance Sheet  Income Statement
2018-06-30 Alphabet Gross Income/Profit            NaN           18875.0
                    Sales/Revenue                  NaN           32758.0
                    Total Assets              197295.0               NaN
                    Total Liabilities          44793.0               NaN
           Amazon   Gross Income/Profit            NaN           22254.0
                    Sales/Revenue                  NaN           52886.0
                    Total Assets              134100.0               NaN
                    Total Liabilities          99105.0               NaN
           Apple    Gross Income/Profit            NaN           20789.0
                    Sales/Revenue                  NaN           53427.0
                    Total Assets              375319.0               NaN
                    Total Liabilities         241272.0               NaN
2018-03-31 Alphabet Gross Income/Profit            NaN           17926.0
                    Sales/Revenue                  NaN           31393.0
                    Total Assets              167497.0               NaN
                    Total Liabilities          28461.0               NaN
           Amazon   Gross Income/Profit            NaN           20307.0
                    Sales/Revenue                  NaN           51042.0
                    Total Assets              126362.0               NaN
                    Total Liabilities          94899.0               NaN
           Apple    Gross Income/Profit            NaN           23530.0
                    Sales/Revenue                  NaN           61224.0
                    Total Assets              321686.0               NaN
                    Total Liabilities         193437.0               NaN
2017-12-31 Alphabet Gross Income/Profit            NaN           18254.0
                    Sales/Revenue                  NaN           32521.0
                    Total Assets              147461.0               NaN
                    Total Liabilities          27130.0               NaN
           Amazon   Gross Income/Profit            NaN           21959.0
                    Sales/Revenue                  NaN           60453.0
                    Total Assets              131310.0               NaN
                    Total Liabilities         103601.0               NaN
           Apple    Gross Income/Profit            NaN           34069.0
                    Sales/Revenue                  NaN           88477.0
                    Total Assets              290479.0               NaN
                    Total Liabilities         171124.0               NaN
2017-09-30 Alphabet Gross Income/Profit            NaN           16815.0
                    Sales/Revenue                  NaN           27963.0
                    Total Assets              129187.0               NaN
                    Total Liabilities          25327.0               NaN
           Amazon   Gross Income/Profit            NaN           16195.0
                    Sales/Revenue                  NaN           43744.0
                    Total Assets              115267.0               NaN
                    Total Liabilities          90609.0               NaN
           Apple    Gross Income/Profit            NaN           19902.0
                    Sales/Revenue                  NaN           52574.0
                    Total Assets              231839.0               NaN
                    Total Liabilities         120292.0               NaN
2017-06-30 Alphabet Gross Income/Profit            NaN           15634.0
                    Sales/Revenue                  NaN           26007.0
                    Total Assets              110920.0               NaN
                    Total Liabilities          23611.0               NaN
           Amazon   Gross Income/Profit            NaN           14504.0
                    Sales/Revenue                  NaN           37955.0
                    Total 

## <code>.swaplevel()</code> and <code>.sort_index()</code>

These methods are used to rearrange the order of the levels on an axis or sort the data by the values in one specific level. 

* .swaplevel() : The swaplevel takes two level numbers or names and returns a new object with the levels interchanged. 
* .sort_index() : This method sorts the data using only the values in a single level. <br>
** Note: ** When swapping levels, it is not uncommon to also use sort_index so that the result is lexicographically sorted by the indicated level. For instance, <code>.swaplevel(0,1).sort_index(level=0)</code>. 


In [71]:
findata3 = findata2.stack(level=1)
findata3

Balance Sheet                      \
                                    Alphabet    Amazon     Apple   
2018-06-30 Gross Income/Profit           NaN       NaN       NaN   
           Sales/Revenue                 NaN       NaN       NaN   
           Total Assets             197295.0  134100.0  375319.0   
           Total Liabilities         44793.0   99105.0  241272.0   
2018-03-31 Gross Income/Profit           NaN       NaN       NaN   
           Sales/Revenue                 NaN       NaN       NaN   
           Total Assets             167497.0  126362.0  321686.0   
           Total Liabilities         28461.0   94899.0  193437.0   
2017-12-31 Gross Income/Profit           NaN       NaN       NaN   
           Sales/Revenue                 NaN       NaN       NaN   
           Total Assets             147461.0  131310.0  290479.0   
           Total Liabilities         27130.0  103601.0  171124.0   
2017-09-30 Gross Income/Profit           NaN       NaN       NaN   
           Sales/Revenue                 NaN       NaN       NaN   
           Total Assets             129187.0  115267.0  231839.0   
           Total Liabilities         25327.0   90609.0  120292.0   
2017-06-30 Gross Income/Profit           NaN       NaN       NaN   
           Sales/Revenue                 NaN       NaN       NaN   
           Total Assets             110920.0   87781.0  207000.0   
           Total Liabilities         23611.0   64567.0   83451.0   

                               Income Statement                    
                                       Alphabet   Amazon    Apple  
2018-06-30 Gross Income/Profit          18875.0  22254.0  20789.0  
           Sales/Revenue                32758.0  52886.0  53427.0  
           Total Assets                     NaN      NaN      NaN  
           Total Liabilities                NaN      NaN      NaN  
2018-03-31 Gross Income/Profit          17926.0  20307.0  23530.0  
           Sales/Revenue                31393.0  51042.0  61224.0  
           Total Assets                     NaN      NaN      NaN  
           Total Liabilities                NaN      NaN      NaN  
2017-12-31 Gross Income/Profit          18254.0  21959.0  34069.0  
           Sales/Revenue                32521.0  60453.0  88477.0  
           Total Assets                     NaN      NaN      NaN  
           Total Liabilities                NaN      NaN      NaN  
2017-09-30 Gross Income/Profit          16815.0  16195.0  19902.0  
           Sales/Revenue                27963.0  43744.0  52574.0  
           Total Assets                     NaN      NaN      NaN  
           Total Liabilities                NaN      NaN      NaN  
2017-06-30 Gross Income/Profit          15634.0  14504.0  17267.0  
           Sales/Revenue                26007.0  37955.0  45260.0  
           Total Assets                     NaN      NaN      NaN  
           Total Liabilities                NaN      NaN      NaN

In [73]:
findata3.stack().swaplevel(0,1)

Balance Sheet  Income Statement
Gross Income/Profit 2018-06-30 Alphabet            NaN           18875.0
                               Amazon              NaN           22254.0
                               Apple               NaN           20789.0
Sales/Revenue       2018-06-30 Alphabet            NaN           32758.0
                               Amazon              NaN           52886.0
                               Apple               NaN           53427.0
Total Assets        2018-06-30 Alphabet       197295.0               NaN
                               Amazon         134100.0               NaN
                               Apple          375319.0               NaN
Total Liabilities   2018-06-30 Alphabet        44793.0               NaN
                               Amazon          99105.0               NaN
                               Apple          241272.0               NaN
Gross Income/Profit 2018-03-31 Alphabet            NaN           17926.0
                               Amazon              NaN           20307.0
                               Apple               NaN           23530.0
Sales/Revenue       2018-03-31 Alphabet            NaN           31393.0
                               Amazon              NaN           51042.0
                               Apple               NaN           61224.0
Total Assets        2018-03-31 Alphabet       167497.0               NaN
                               Amazon         126362.0               NaN
                               Apple          321686.0               NaN
Total Liabilities   2018-03-31 Alphabet        28461.0               NaN
                               Amazon          94899.0               NaN
                               Apple          193437.0               NaN
Gross Income/Profit 2017-12-31 Alphabet            NaN           18254.0
                               Amazon              NaN           21959.0
                               Apple               NaN           34069.0
Sales/Revenue       2017-12-31 Alphabet            NaN           32521.0
                               Amazon              NaN           60453.0
                               Apple               NaN           88477.0
Total Assets        2017-12-31 Alphabet       147461.0               NaN
                               Amazon         131310.0               NaN
                               Apple          290479.0               NaN
Total Liabilities   2017-12-31 Alphabet        27130.0               NaN
                               Amazon         103601.0               NaN
                               Apple          171124.0               NaN
Gross Income/Profit 2017-09-30 Alphabet            NaN           16815.0
                               Amazon              NaN           16195.0
                               Apple               NaN           19902.0
Sales/Revenue       2017-09-30 Alphabet            NaN           27963.0
                               Amazon              NaN           43744.0
                               Apple               NaN           52574.0
Total Assets        2017-09-30 Alphabet       129187.0               NaN
                               Amazon         115267.0               NaN
                               Apple          231839.0               NaN
Total Liabilities   2017-09-30 Alphabet        25327.0               NaN
                               Amazon          90609.0               NaN
                               Apple          120292.0               NaN
Gross Income/Profit 2017-06-30 Alphabet            NaN           15634.0
                               Amazon              NaN           14504.0
                               Apple               NaN           17267.0
Sales/Revenue       2017-06-30 Alphabet            NaN           26007.0
                               Amazon              NaN           37955.0
                               Apple               NaN           45260.0
Total Assets        2017-0

In [74]:
findata3.stack().swaplevel(0,1).sort_index(level=0)

Balance Sheet  Income Statement
Gross Income/Profit 2017-06-30 Alphabet            NaN           15634.0
                               Amazon              NaN           14504.0
                               Apple               NaN           17267.0
                    2017-09-30 Alphabet            NaN           16815.0
                               Amazon              NaN           16195.0
                               Apple               NaN           19902.0
                    2017-12-31 Alphabet            NaN           18254.0
                               Amazon              NaN           21959.0
                               Apple               NaN           34069.0
                    2018-03-31 Alphabet            NaN           17926.0
                               Amazon              NaN           20307.0
                               Apple               NaN           23530.0
                    2018-06-30 Alphabet            NaN           18875.0
                               Amazon              NaN           22254.0
                               Apple               NaN           20789.0
Sales/Revenue       2017-06-30 Alphabet            NaN           26007.0
                               Amazon              NaN           37955.0
                               Apple               NaN           45260.0
                    2017-09-30 Alphabet            NaN           27963.0
                               Amazon              NaN           43744.0
                               Apple               NaN           52574.0
                    2017-12-31 Alphabet            NaN           32521.0
                               Amazon              NaN           60453.0
                               Apple               NaN           88477.0
                    2018-03-31 Alphabet            NaN           31393.0
                               Amazon              NaN           51042.0
                               Apple               NaN           61224.0
                    2018-06-30 Alphabet            NaN           32758.0
                               Amazon              NaN           52886.0
                               Apple               NaN           53427.0
Total Assets        2017-06-30 Alphabet       110920.0               NaN
                               Amazon          87781.0               NaN
                               Apple          207000.0               NaN
                    2017-09-30 Alphabet       129187.0               NaN
                               Amazon         115267.0               NaN
                               Apple          231839.0               NaN
                    2017-12-31 Alphabet       147461.0               NaN
                               Amazon         131310.0               NaN
                               Apple          290479.0               NaN
                    2018-03-31 Alphabet       167497.0               NaN
                               Amazon         126362.0               NaN
                               Apple          321686.0               NaN
                    2018-06-30 Alphabet       197295.0               NaN
                               Amazon         134100.0               NaN
                               Apple          375319.0               NaN
Total Liabilities   2017-06-30 Alphabet        23611.0               NaN
                               Amazon          64567.0               NaN
                               Apple           83451.0               NaN
                    2017-09-30 Alphabet        25327.0               NaN
                               Amazon          90609.0               NaN
                               Apple          120292.0               NaN
                    2017-12-31 Alphabet        27130.0               NaN
                               Amazon         103601.0               NaN
                               Apple          171124.0               NaN
                    2018-0

# Aggregate, Group, Filter and Transform Data

Aggregation Methods:
* df.count() : This Method counts total number of items
* df.first(), df.last() : Methods to get first and last item respectively.
* df.mean(), df.median() : Methods to get mean and median respectively.
* df. min(), df.max() : Methods to get minimum and maximum values respectively.
* df.std(), df.var(): Methods to get standard deviation and variance respectively.
* df.mad() : Method to get mean absolute deviation 
* df.prod() : Method to get product of the all items
* df.sum() : Method to get sum of all the items.

Groupby: Split, Apply ,Combine
* Split : Breaks and groups data frame depending on the value of the specified key.
* Apply : Computes some functions, usually an aggregate, transformation or filtering, within individual groups.
* Combine : Merges the results of these operations into an output array.

In [166]:
import numpy as np
rng = np.random.RandomState(0)
df = pd.DataFrame({'key' : [ 'A', 'B', 'C', 'A', 'B', 'C'], 'data1': range(6), 'data2': rng.randint(0, 10, 6)})
df


data1  data2 key
0      0      5   A
1      1      0   B
2      2      3   C
3      3      3   A
4      4      7   B
5      5      9   C

## Grouping using <code>.groupby()</code>

In [89]:
#Grouping by key using the mean to aggregate
df.groupby('key').mean()

data1  data2
key              
A      1.5    4.0
B      2.5    3.5
C      3.5    6.0

In [111]:
import seaborn as sns # statistical data visualization
titanic = sns.load_dataset('titanic') # load_dataset looks for online csv files 
                                    #on https://github.com/mwaskom/seaborn-data
titanic.head()

survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True

In [112]:
#Grouping by sex using the mean to aggregate
titanic.groupby('sex').sum()

survived  pclass       age  sibsp  parch        fare  adult_male  \
sex                                                                        
female       233     678   7286.00    218    204  13966.6628         0.0   
male         109    1379  13919.17    248    136  14727.2865       537.0   

        alone  
sex            
female  126.0  
male    411.0

In [119]:
#Grouping by class using the mean to aggregate
titanic.groupby('class').mean()

survived  pclass        age     sibsp     parch       fare  \
class                                                                
First   0.629630     1.0  38.233441  0.416667  0.356481  84.154687   
Second  0.472826     2.0  29.877630  0.402174  0.380435  20.662183   
Third   0.242363     3.0  25.140620  0.615071  0.393075  13.675550   

        adult_male     alone  
class                         
First     0.550926  0.504630  
Second    0.538043  0.565217  
Third     0.649695  0.659878

## Looking at the groups with <code>.get_group()</code>

Gruping data by 'sex' and accessing the group 'female'

In [131]:
titanic.groupby('sex').get_group('female').head()

survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
1         1       1  female  38.0      1      0  71.2833        C   First   
2         1       3  female  26.0      0      0   7.9250        S   Third   
3         1       1  female  35.0      1      0  53.1000        S   First   
8         1       3  female  27.0      0      2  11.1333        S   Third   
9         1       2  female  14.0      1      0  30.0708        C  Second   

     who  adult_male deck  embark_town alive  alone  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
8  woman       False  NaN  Southampton   yes  False  
9  child       False  NaN    Cherbourg   yes  False

Grouping by 'sex' and 'class' and accessing the group 'female' 'First'

In [142]:
titanic.groupby(['sex','class']).get_group(('female','First')).head()

survived  pclass     sex   age  sibsp  parch      fare embarked  class  \
1          1       1  female  38.0      1      0   71.2833        C  First   
3          1       1  female  35.0      1      0   53.1000        S  First   
11         1       1  female  58.0      0      0   26.5500        S  First   
31         1       1  female   NaN      1      0  146.5208        C  First   
52         1       1  female  49.0      1      0   76.7292        C  First   

      who  adult_male deck  embark_town alive  alone  
1   woman       False    C    Cherbourg   yes  False  
3   woman       False    C  Southampton   yes  False  
11  woman       False    C  Southampton   yes   True  
31  woman       False    B    Cherbourg   yes  False  
52  woman       False    D    Cherbourg   yes  False

Now aggregating this group using the mean and corroborating the plot of the movie 'Titanic'

In [147]:
titanic.groupby(['sex','class']).get_group(('female','First')).mean()

survived        0.968085
pclass          1.000000
age            34.611765
sibsp           0.553191
parch           0.457447
fare          106.125798
adult_male      0.000000
alone           0.361702
dtype: float64

In [152]:
titanic.groupby(['sex','class']).get_group(('male','Third')).mean()

survived       0.135447
pclass         3.000000
age           26.507589
sibsp          0.498559
parch          0.224784
fare          12.661633
adult_male     0.919308
alone          0.760807
dtype: float64

## <code>.agg</code> (short for aggregate) affords more control 
Aggregation works for all columns

In [113]:
#Aggregation
titanic.groupby('survived').agg('min') # for instance look at min age of who survived


pclass     sex   age  sibsp  parch  fare embarked    who  \
survived                                                             
0              1  female  1.00      0      0   0.0        C  child   
1              1  female  0.42      0      0   0.0      inf  child   

          adult_male embark_town alive  alone  
survived                                       
0              False   Cherbourg    no  False  
1              False         inf   yes  False

Multiple aggregations

In [150]:
#Aggregation multiple
titanic.groupby('survived').agg(['min','max']) # for instance look at min max age of who survived


pclass         sex         age       sibsp     parch      ...   \
            min max     min   max   min   max   min max   min max  ...    
survived                                                           ...    
0             1   3  female  male  1.00  74.0     0   8     0   6  ...    
1             1   3  female  male  0.42  80.0     0   4     0   5  ...    

            who        adult_male       embark_town              alive       \
            min    max        min   max         min          max   min  max   
survived                                                                      
0         child  woman      False  True   Cherbourg  Southampton    no   no   
1         child  woman      False  True         inf  Southampton   yes  yes   

          alone        
            min   max  
survived               
0         False  True  
1         False  True  

[2 rows x 24 columns]

Aggregating a single column, age

In [118]:
#Aggregation of column
titanic.groupby('survived').age.agg(['min','mean','max']) # age: min, mean, max


min       mean   max
survived                       
0         1.00  30.626179  74.0
1         0.42  28.343690  80.0

Different aggregations per column

In [127]:
#Aggregation of column
titanic.groupby('survived').agg({'age':['min','mean','max'],'fare':'mean'}) # age: min, mean, max

fare   age                 
               mean   min       mean   max
survived                                  
0         22.117887  1.00  30.626179  74.0
1         48.395408  0.42  28.343690  80.0

## Tranformation using <code>.transform</code>
While aggregation must return a reduced version of the data, transformation can operate on entire grouped data but one group at a time. In the example below, the mean of a particular group is substracted from each element of the group.

In [164]:
# Transformation

titanic.groupby('sex').transform(lambda x: x - x.mean()).head()

survived    pclass        age     sibsp     parch       fare  adult_male  \
0 -0.188908  0.610052  -8.726645  0.570191 -0.235702 -18.273893    0.069324   
1  0.257962 -1.159236  10.084291  0.305732 -0.649682  26.803482    0.000000   
2  0.257962  0.840764  -1.915709 -0.694268 -0.649682 -36.554818    0.000000   
3  0.257962 -1.159236   7.084291  0.305732 -0.649682   8.620182    0.000000   
4 -0.188908  0.610052   4.273355 -0.429809 -0.235702 -17.473893    0.069324   

      alone  
0 -0.712305  
1 -0.401274  
2  0.598726  
3 -0.401274  
4  0.287695

## <code>Apply()</code>

This method lets us apply an arbitrary function to the group results. This method should take a data frame and return either a Pandas object (e.g., DataFrame or Series) or a scalar.



In [167]:
df

data1  data2 key
0      0      5   A
1      1      0   B
2      2      3   C
3      3      3   A
4      4      7   B
5      5      9   C

In [170]:
def norm_by_data2(x):
    x['data1'] /= x['data2'].sum()
    return x

df.groupby('key').get_group('A')
# in example below, A[data1]=A[data1]/A[data2].sum, results in [0] 0/8=0 and [3] 3/8=0.375 


data1  data2 key
0      0      5   A
3      3      3   A

In [172]:
df.groupby('key').get_group('B')
# in example below, B[data1]=B[data1]/B[data2].sum, results in [1] 1/7=0.14 and [4] 4/7=0.57

data1  data2 key
1      1      0   B
4      4      7   B

In [173]:
df.groupby('key').apply(norm_by_data2)

data1  data2 key
0  0.000000      5   A
1  0.142857      0   B
2  0.166667      3   C
3  0.375000      3   A
4  0.571429      7   B
5  0.416667      9   C

# Merging data with <code>.merge()</code>

To combine datasets by linking rows using one or more keys, we use <code>.merge()</code> operation. <br>

In [177]:
df1 = pd.DataFrame({'key1': ['b', 'b', 'a', 'c', 'a', 'a',  'b'], 'data1': range(7)})
df1

data1 key1
0      0    b
1      1    b
2      2    a
3      3    c
4      4    a
5      5    a
6      6    b

In [178]:
df2 = pd.DataFrame ({'key1': [ 'a', 'b', 'd'], 'data2': range(3)})
df2

data2 key1
0      0    a
1      1    b
2      2    d

In [179]:
pd.merge(df1, df2) 

data1 key1  data2
0      0    b      1
1      1    b      1
2      6    b      1
3      2    a      0
4      4    a      0
5      5    a      0

This <code>.merge()</code> merges by using the overlapping column names as the keys, <code>key1</code> in this case. This is because we did not specify which column to **join** explicitly. We can specify which column is the key for the join as follows.



In [181]:
pd.merge(df1, df2, on = 'key1')


data1 key1  data2
0      0    b      1
1      1    b      1
2      6    b      1
3      2    a      0
4      4    a      0
5      5    a      0

By default the behaviour of <code>.merge()</code> is to conduct an __inner join__, however that can also be modified as follows:

Other options : <br>
* <code>how = 'inner' </code>: Use only the key combinations observed in both tables.
* <code>how = 'outer' </code>: Use all the key combinations observed in both tables together.
* <code>how = 'left' </code>: Use all key combinations found in the left table.
* <code>how = 'right' </code> : Use all key combinations found in the right table.


In [182]:
pd.merge(df1, df2, how = 'outer')

data1 key1  data2
0    0.0    b    1.0
1    1.0    b    1.0
2    6.0    b    1.0
3    2.0    a    0.0
4    4.0    a    0.0
5    5.0    a    0.0
6    3.0    c    NaN
7    NaN    d    2.0

In [99]:
pd.merge(df1, df2, how = 'right')

data1 key  data2
0    0.0   b      1
1    1.0   b      1
2    6.0   b      1
3    2.0   a      0
4    4.0   a      0
5    5.0   a      0
6    NaN   d      2

# Pivot Tables

A pivot table is a similar operation that is commonly seen in spreadsheets and other programs that operate on tabular data. The pivot table takes simple column-wise data as input and groups the entries into a two dimensional table that provides a multidimensional summarization of the data. <br>

In [58]:
import seaborn as sns # statistical data visualization
titanic = sns.load_dataset('titanic') # load_dataset looks for online csv files 
                                    #on https://github.com/mwaskom/seaborn-data
titanic

survived  pclass     sex   age  sibsp  parch      fare embarked   class  \
0           0       3    male  22.0      1      0    7.2500        S   Third   
1           1       1  female  38.0      1      0   71.2833        C   First   
2           1       3  female  26.0      0      0    7.9250        S   Third   
3           1       1  female  35.0      1      0   53.1000        S   First   
4           0       3    male  35.0      0      0    8.0500        S   Third   
5           0       3    male   NaN      0      0    8.4583        Q   Third   
6           0       1    male  54.0      0      0   51.8625        S   First   
7           0       3    male   2.0      3      1   21.0750        S   Third   
8           1       3  female  27.0      0      2   11.1333        S   Third   
9           1       2  female  14.0      1      0   30.0708        C  Second   
10          1       3  female   4.0      1      1   16.7000        S   Third   
11          1       1  female  58.0      0      0   26.5500        S   First   
12          0       3    male  20.0      0      0    8.0500        S   Third   
13          0       3    male  39.0      1      5   31.2750        S   Third   
14          0       3  female  14.0      0      0    7.8542        S   Third   
15          1       2  female  55.0      0      0   16.0000        S  Second   
16          0       3    male   2.0      4      1   29.1250        Q   Third   
17          1       2    male   NaN      0      0   13.0000        S  Second   
18          0       3  female  31.0      1      0   18.0000        S   Third   
19          1       3  female   NaN      0      0    7.2250        C   Third   
20          0       2    male  35.0      0      0   26.0000        S  Second   
21          1       2    male  34.0      0      0   13.0000        S  Second   
22          1       3  female  15.0      0      0    8.0292        Q   Third   
23          1       1    male  28.0      0      0   35.5000        S   First   
24          0       3  female   8.0      3      1   21.0750        S   Third   
25          1       3  female  38.0      1      5   31.3875        S   Third   
26          0       3    male   NaN      0      0    7.2250        C   Third   
27          0       1    male  19.0      3      2  263.0000        S   First   
28          1       3  female   NaN      0      0    7.8792        Q   Third   
29          0       3    male   NaN      0      0    7.8958        S   Third   
..        ...     ...     ...   ...    ...    ...       ...      ...     ...   
861         0       2    male  21.0      1      0   11.5000        S  Second   
862         1       1  female  48.0      0      0   25.9292        S   First   
863         0       3  female   NaN      8      2   69.5500        S   Third   
864         0       2    male  24.0      0      0   13.0000        S  Second   
865         1       2  female  42.0      0      0   13.0000        S  Second   
866         1       2  female  27.0      1      0   13.8583        C  Second   
867         0       1    male  31.0      0      0   50.4958        S   First   
868         0       3    male   NaN      0      0    9.5000        S   Third   
869         1       3    male   4.0      1      1   11.1333        S   Third   
870         0       3    male  26.0      0      0    7.8958        S   Third   
871         1       1  female  47.0      1      1   52.5542        S   First   
872         0       1    male  33.0      0      0    5.0000        S   First   
873         0       3    male  47.0      0      0    9.0000        S   Third   
874         1       2  female  28.0      1      0   24.0000        C  Second   
875         1       3  female  15.0      0      0    7.2250        C   Third   
876         0       3    male  20.0      0      0    9.8458        S   Third   
877         0       3    male  19.0      0      0    7.8958        S   Third   
878         0       3    male   NaN      0      0    7.8958        S   Third   
879  

In [183]:
titanic.pivot_table(values=['survived','fare'], index = 'sex', columns = 'class')

fare                        survived                    
class        First     Second      Third     First    Second     Third
sex                                                                   
female  106.125798  21.970121  16.118810  0.968085  0.921053  0.500000
male     67.226127  19.741782  12.661633  0.368852  0.157407  0.135447

The aggfunc keyword controls what type of aggregation is applied, which is mean by default. As in the GroupBy, the aggregation specification can be a string representing one of several common choices(‘sum’, ‘mean’, ‘count’, ‘min’, ‘max’, etc.) or a function that implements an aggregation (np.sum(), min(), sum(), etc.).

Additionally, it can be specified as dictionary mapping a column to any of the above desired options: 

In [184]:
titanic.pivot_table(index = 'sex', columns = 'class', aggfunc = {'survived' : sum, 'fare' : 'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

# Data cleaning and preparation

Data cleaning is the process of removing bad data in a dataset. This bad data includes incorrect and improperly formatted data as well as duplicated and missing data. 

In [190]:
# Example of a student survey dataset which includes incorrect and improperly formatted data.

csv_path = 'https://s3.amazonaws.com/dss-fall2018/Student_Survey.csv'

df = pd.read_csv (csv_path)
df

Year    Location      Education  Sample_Size Satisfactory
0  2017.0      Putnam  Middle School        659.0            Y
1  2018.0   Lexington  Middle School        649.0            N
2  2018.0   Lexington  Middle School        435.0            N
3  2017.0    Berkeley            NaN          NaN          NaN
4     NaN    Berkeley    High School        228.0            Y
5  2018.0    Berkeley  Middle School         20.0          NaN
6  2018.0  Washington    High School        437.0            N
7     NaN     Tremont    High School          NaN            Y
8  2016.0     Tremont    High School        220.0            Y

Let's take a look at the dataset. There are seven NA values in all columns. By using ```isnull``` function, pandas recognizes all missing value and return ```True```  

In [186]:
# Recognizing missing values

print (df.isnull())


    Year  Location  Education  Sample_Size  Satisfactory
0  False     False      False        False         False
1  False     False      False        False         False
2  False     False      False        False         False
3  False     False       True         True          True
4   True     False      False        False         False
5  False     False      False        False          True
6  False     False      False        False         False
7   True     False      False         True         False
8  False     False      False        False         False


### ```dropna``` method

```dropna``` is a method to filter missing data. Sometimes you need to work on only correct data and want to omit others.  

In [189]:
# Dropping all missing data by omitting rows and columns which include missing data

df.dropna()

Year    Location      Education  Sample_Size Satisfactory
0  2017.0      Putnam  Middle School        659.0            Y
1  2018.0   Lexington  Middle School        649.0            N
2  2018.0   Lexington  Middle School        435.0            N
6  2018.0  Washington    High School        437.0            N
8  2016.0     Tremont    High School        220.0            Y

In [188]:
# Dropping rows and columns which are all NA (In this example, there is no row includes all NA)

df.dropna(how = 'all')    #For rows

Year    Location      Education  Sample_Size Satisfactory
0  2017.0      Putnam  Middle School        659.0            Y
1  2018.0   Lexington  Middle School        649.0            N
2  2018.0   Lexington  Middle School        435.0            N
3  2017.0    Berkeley            NaN          NaN          NaN
4     NaN    Berkeley    High School        228.0            Y
5  2018.0    Berkeley  Middle School         20.0          NaN
6  2018.0  Washington    High School        437.0            N
7     NaN     Tremont    High School          NaN            Y
8  2016.0     Tremont    High School        220.0            Y

In [187]:
df.dropna(axis = 1, how ='all')     #For columns

Year    Location      Education  Sample_Size Satisfactory
0  2017.0      Putnam  Middle School        659.0            Y
1  2018.0   Lexington  Middle School        649.0            N
2  2018.0   Lexington  Middle School        435.0            N
3  2017.0    Berkeley            NaN          NaN          NaN
4     NaN    Berkeley    High School        228.0            Y
5  2018.0    Berkeley  Middle School         20.0          NaN
6  2018.0  Washington    High School        437.0            N
7     NaN     Tremont    High School          NaN            Y
8  2016.0     Tremont    High School        220.0            Y

### ```fillna``` method 

```fillna``` is a method to fill missing data by any number or value.  

In [195]:
# filling missing data by '100' in 'Sample_Size' column. 

print (df['Sample_Size'])
df.fillna({'Sample_Size': 100})

0    659.0
1    649.0
2    435.0
3      NaN
4    228.0
5     20.0
6    437.0
7      NaN
8    220.0
Name: Sample_Size, dtype: float64


Year    Location      Education  Sample_Size Satisfactory
0  2017.0      Putnam  Middle School        659.0            Y
1  2018.0   Lexington  Middle School        649.0            N
2  2018.0   Lexington  Middle School        435.0            N
3  2017.0    Berkeley            NaN        100.0          NaN
4     NaN    Berkeley    High School        228.0            Y
5  2018.0    Berkeley  Middle School         20.0          NaN
6  2018.0  Washington    High School        437.0            N
7     NaN     Tremont    High School        100.0            Y
8  2016.0     Tremont    High School        220.0            Y

In [198]:
# filling missing data in both columns 'Year' by 2015 and 'Sample_Size' by '100'.

df.fillna({'Year': 2015, 'Sample_Size': 100})

Year    Location      Education  Sample_Size Satisfactory
0  2017.0      Putnam  Middle School        659.0            Y
1  2018.0   Lexington  Middle School        649.0            N
2  2018.0   Lexington  Middle School        435.0            N
3  2017.0    Berkeley            NaN        100.0          NaN
4  2015.0    Berkeley    High School        228.0            Y
5  2018.0    Berkeley  Middle School         20.0          NaN
6  2018.0  Washington    High School        437.0            N
7  2015.0     Tremont    High School        100.0            Y
8  2016.0     Tremont    High School        220.0            Y

## Removing Duplicates

Sometimes in a DataFrame, you have duplicate rows and you need to remove them. See dataframe below with few duplicate rows.

In [203]:
# Creating a DataFrame with few duplicated rows
raw_data = [['Clark','Kent',28,3,70],['Bruce','Banner',38,4,25],['Clark','Kent',28,3,70],['Tony','Stark',42,24,94],['Hal','Jordan',25,31,57],['Bruce','Wayne',32,2,62],
            ['Tony','Stark',42,24,94],['Tony','Stark',42,24,94]]
df = pd.DataFrame (raw_data, columns = ['first_name', 'last_name','age','preTestScore','postTestScore'])
df

first_name last_name  age  preTestScore  postTestScore
0      Clark      Kent   28             3             70
1      Bruce    Banner   38             4             25
2      Clark      Kent   28             3             70
3       Tony     Stark   42            24             94
4        Hal    Jordan   25            31             57
5      Bruce     Wayne   32             2             62
6       Tony     Stark   42            24             94
7       Tony     Stark   42            24             94

```duplicated()``` method returns a boolean value whether each row is a duplicate. In this DataFrame, the value for number 5 is 'True' and says row number 5 is duplicated.

In [204]:
df.duplicated()

0    False
1    False
2     True
3    False
4    False
5    False
6     True
7     True
dtype: bool

```drop_duplicates()``` method returns DataFrame with duplicate rows removed

In [208]:
df.drop_duplicates()

first_name last_name  age  preTestScore  postTestScore
0      Clark      Kent   28             3             70
1      Bruce    Banner   38             4             25
3       Tony     Stark   42            24             94
4        Hal    Jordan   25            31             57
5      Bruce     Wayne   32             2             62

# References
1. Seaborn, statistical data visualization, https://seaborn.pydata.org
2. Data repo for Seaborn examples, https://github.com/mwaskom/seaborn-data
3. Pivot Tables, https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html
4. Reshaping, Pivot, Stack, unstack, https://pandas.pydata.org/pandas-docs/stable/reshaping.html

_Last updated on 9.20.18 2:24am<br>
(C) 2018 Complex Adaptive Systems Laboratory all rights reserved._